In [1]:
from fastai.vision import *
from bs4 import BeautifulSoup
import requests
import re
from urllib.request import urlopen
import urllib.request
import os
import json
import pandas as pd
import csv
import math

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import random
import cv2
import matplotlib.pyplot as plt

In [12]:
def read_image(path):
    im = cv2.imread(str(path))
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

def normalize(im):
    """Normalizes images with Imagenet stats."""
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im - imagenet_stats[0])/imagenet_stats[1]

def apply_transforms(x, sz=(303, 227), zoom=1.05):
    """ Applies a random crop, rotation"""
    sz1 = int(zoom*sz[0])
    sz2 = int(zoom*sz[1])
    x = cv2.resize(x, (sz1, sz2))
    x = rotate_cv(x, np.random.uniform(-10,10))
    x = random_crop(x, sz[1], sz[0])
    if np.random.rand() >= .5:
                x = np.fliplr(x).copy()
    return x

In [4]:
class imageDataset(Dataset):
    def __init__(self, files_path, labels, transform=False):
        self.num_files = len(labels)
        self.files_path = files_path
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return self.num_files
    
    def __getitem__(self, index):
        y = self.labels[index]
        file_name =os.listdir(self.files_path)[index]
        path = self.files_path/str(file_name)
        x = read_image(path)
        if self.transform:
            x = apply_transforms(x)
        else:
            x = cv2.resize(x, (303, 227))
        x = normalize(x)
        x = np.rollaxis(x, 2)
        return x, y

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        resnet = models.resnet34(pretrained=True)
        # freezing parameters
        for param in resnet.parameters():
            param.requires_grad = False
        # convolutional layers of resnet34
        layers = list(resnet.children())[:8]
        self.top_model = nn.Sequential(*layers)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 5)
    
    def forward(self, x):
        x = F.relu(self.top_model(x))
        x = nn.AdaptiveAvgPool2d((1,1))(x)
        x = x.view(x.shape[0], -1) # flattening 
        #x = nn.Dropout(0.2)(x)
        x = self.bn1(x)
        x = F.relu(self.fc1(x))
        #x = nn.Dropout(0.2)(x)
        x = self.bn2(x)
        x = self.fc2(x)
        return x

In [6]:
def downloadImg(imgPath):
    '''
    downloads the image from s3. Not sure if we need this function.
    '''

In [13]:
def tagImg(img,model,tagMap):
    output=model(img.float())
    _, pred = torch.max(output.data, 1)
    pred=pred.item()
    tag=tagMap[pred]
    return tag

In [14]:
PATH = Path("")
#Folder location from where model will pick images for tagging 
pathTestImg= PATH/'new_test/'
#Path where the model is saved
modelPath=PATH/'fullModel.pt'
#tag mapping 
tagMap={0:'car', 1:'dress', 2:'hairstyle',3:'art', 4:'watch'}
#load model
model=torch.load(modelPath)

In [15]:
#dataloader object needs true labels. For test data we dont have labels but we cannot give empty list.
# Hence we have just pass fake labels, the model wont use them to predict.
# Assuming that model predicts tag for single image at a time. fake label is [4]
testds=imageDataset(pathTestImg,[4])
test_dl=DataLoader(testds)

for img,label in test_dl:
    tag=tagImg(img,model,tagMap)
    print('Tag is',tag)   

Tag is car
